<a href="https://colab.research.google.com/github/pabrey/datasciencecoursera/blob/master/4b_all_routes_mean_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previous

In [1]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
import json
from google.colab import files
import time
from datetime import datetime
import math

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create functions

## load data

In [0]:

def load_data(location):
  Separator = ';'

  df =  pd.read_csv(location, sep=Separator)

  df.columns = ["CASE_ID", "ACTIVITY",	"TIMESTAMP_IN",	"TIMESTAMP_OUT", "COLOR"] 
  df["DIFFERENCE"]= df["TIMESTAMP_OUT"] - df["TIMESTAMP_IN"]
  df_order = df.sort_values(['CASE_ID', 'TIMESTAMP_IN'])

  print(df_order.head())
  print(df_order.head(1))
  print(df_order.tail(1))
  print(df_order.tail(1))
  return df_order

## prefilter

thers elements with timestamp to 0, delete this elements

In [0]:
def filter_bad_data(df_input):
  #hers elements with timestamp to 0, delete this elements
  id_filter = df_input[df_input["TIMESTAMP_IN"] == 0]["CASE_ID"]
  df_filter_out = df_input[~df_input["CASE_ID"].isin(id_filter)]
  id_filter = df_input[df_input["TIMESTAMP_OUT"] == 0]["CASE_ID"]
  df_filter_out = df_filter_out[~df_filter_out["CASE_ID"].isin(id_filter)]
  df_filter_out = df_filter_out.sort_values(['CASE_ID', 'TIMESTAMP_IN'])
  
  #elementos no ordenados
  df_unorder = df_filter_out.sort_values("TIMESTAMP_IN").groupby("CASE_ID").tail(1)[["CASE_ID", "ACTIVITY"]] 
  id_unorder = df_unorder[df_unorder["ACTIVITY"] != "Aalta"]["CASE_ID"]
  df_filter_out = df_filter_out[~df_filter_out["CASE_ID"].isin(id_unorder)]
  
  

  df_filter_out = df_filter_out.sort_values(["CASE_ID", "TIMESTAMP_IN"])
  
  print("Len before prefilter: "+str(len(df_input["CASE_ID"].drop_duplicates())))
  print("Len after prefilter: "+str(len(df_filter_out["CASE_ID"].drop_duplicates())))
  
  return df_filter_out

## create minero

In [0]:

def create_minero_file(input_df, ouput_file):
  input_df = input_df[["CASE_ID", "ACTIVITY", "TIMESTAMP_IN"]]
  input_df = input_df.sort_values(["CASE_ID",  "TIMESTAMP_IN"])
  input_df["DATE_IN"] = (pd.to_datetime(input_df["TIMESTAMP_IN"]*1000,unit='ms'))
  input_df["TIMESTAMP"] = input_df["DATE_IN"].dt.strftime('%d/%m/%Y %H:%M:%S')+ " UTC"

  input_df[["CASE_ID","ACTIVITY","TIMESTAMP"]].sort_values([ "TIMESTAMP"]).to_csv(ouput_file, index=False, sep = ';')
  files.download(ouput_file)

# Test all routes, parameters 00, resources 1

load and filter

In [7]:
location = 'drive/My Drive/data/prosae/testAllRoutesMean/res_simulacion_all_mean_00_1.csv'
df_load = load_data(location)
df_filter =  filter_bad_data(df_load)

   CASE_ID        ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
4        1           Aalta  0.000000e+00   0.000000e+00    NaN  0.000000e+00
0        1        admision  1.614764e+08   1.614772e+08    NaN  8.395910e+02
1        1          triaje  1.614772e+08   1.614816e+08    NaN  4.409802e+03
2        1    rx solicitud  1.614816e+08   1.614831e+08    NaN  1.452060e+03
3        1  rx exploracion  1.614831e+08   0.000000e+00    NaN -1.614831e+08
   CASE_ID ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR  DIFFERENCE
4        1    Aalta           0.0            0.0    NaN         0.0
      CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
1779      961  admision  1.620757e+08            0.0    NaN -1.620757e+08
      CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
1779      961  admision  1.620757e+08            0.0    NaN -1.620757e+08
Len before prefilter: 820
Len after prefilter: 812


check elements whose last element is not alta and first is not admision

In [8]:
(df_filter.sort_values("TIMESTAMP_IN").groupby("CASE_ID")["ACTIVITY"].first() ).drop_duplicates()

CASE_ID
2    admision
Name: ACTIVITY, dtype: object

show results

In [9]:
df_filter.groupby("ACTIVITY")["DIFFERENCE"].mean()

ACTIVITY
Aalta                 0.000000
admision            731.686558
interconsulta     25553.238663
respuesta         14071.090185
rx informe        10396.980809
rx solicitud      19681.638782
triaje           330517.054210
Name: DIFFERENCE, dtype: float64

create file minero

In [0]:
ouput_file= 'input_data_miner_all_mean_00_1.csv'
create_minero_file(df_filter, ouput_file)

#Test all routes,parameters 00 , resources 200

load and filter

In [14]:
location = 'drive/My Drive/data/prosae/testAllRoutesMean/res_simulacion_all_mean_00_200.csv'
df_load = load_data(location)
df_filter =  filter_bad_data(df_load)


   CASE_ID        ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
4        1           Aalta  0.000000e+00   0.000000e+00    NaN  0.000000e+00
0        1        admision  1.614764e+08   1.614772e+08    NaN  8.395910e+02
1        1          triaje  1.614772e+08   1.614816e+08    NaN  4.409802e+03
2        1    rx solicitud  1.614816e+08   1.614900e+08    NaN  8.332127e+03
3        1  rx exploracion  1.614900e+08   0.000000e+00    NaN -1.614900e+08
   CASE_ID ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR  DIFFERENCE
4        1    Aalta           0.0            0.0    NaN         0.0
      CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
3643     2849  admision  1.620761e+08            0.0    NaN -1.620761e+08
      CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
3643     2849  admision  1.620761e+08            0.0    NaN -1.620761e+08
Len before prefilter: 795
Len after prefilter: 525


In [15]:
df_filter =  filter_bad_data(df_load)

Len before prefilter: 795
Len after prefilter: 525


check elements whose last element is not alta and first is not admision

In [16]:
(df_filter.sort_values("TIMESTAMP_IN").groupby("CASE_ID")["ACTIVITY"].first() ).drop_duplicates()

CASE_ID
16    admision
Name: ACTIVITY, dtype: object

show results

In [17]:
df_filter.groupby("ACTIVITY")["DIFFERENCE"].mean()

ACTIVITY
Aalta                0.000000
admision           742.912785
interconsulta    15480.032789
respuesta        17771.039831
rx informe       21655.866931
rx solicitud      6676.397136
triaje            7574.638427
Name: DIFFERENCE, dtype: float64

In [9]:
len(df_filter)

1718

In [10]:
df_filter[df_filter["ACTIVITY"] == "admision"].head()

,CASE_ID,ACTIVITY,TIMESTAMP_IN,TIMESTAMP_OUT,COLOR,DIFFERENCE
5,2,admision,1.614772e+08,1.614780e+08,NaN,769.329164
9,4,admision,1.614780e+08,1.614786e+08,NaN,573.798124
12,5,admision,1.614786e+08,1.614804e+08,NaN,1863.717873
16,6,admision,1.614804e+08,1.614819e+08,NaN,1464.999379
20,9,admision,1.614819e+08,1.614821e+08,NaN,200.117493


## create file minero

In [0]:
ouput_file= 'input_data_miner_all_mean_00_200.csv'
create_minero_file(df_filter, ouput_file)

# Test all routes, parameters 00 resources 200, extended time

load and filter

In [29]:
location = 'drive/My Drive/data/prosae/testAllRoutesMean/res_simulacion_all_mean_00_200_extended_time.csv'
df_load = load_data(location)
df_filter =  filter_bad_data(df_load)


   CASE_ID        ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
0        1        admision  1.614764e+08   1.614772e+08    NaN  8.395910e+02
1        1          triaje  1.614772e+08   1.614816e+08    NaN  4.409802e+03
2        1    rx solicitud  1.614816e+08   1.614900e+08    NaN  8.332127e+03
3        1  rx exploracion  1.614900e+08   1.627409e+08    NaN  1.250893e+06
4        1      rx informe  1.627409e+08   1.627467e+08    NaN  5.860907e+03
   CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR  DIFFERENCE
0        1  admision   161476390.0   1.614772e+08    NaN  839.591004
       CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
37276    29290  admision  1.674755e+08            0.0    NaN -1.674755e+08
       CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR    DIFFERENCE
37276    29290  admision  1.674755e+08            0.0    NaN -1.674755e+08
Len before prefilter: 7986
Len after prefilter: 7345


check elements whose last element is not alta and first is not admision

In [30]:
(df_filter.sort_values("TIMESTAMP_IN").groupby("CASE_ID")["ACTIVITY"].first() ).drop_duplicates()

CASE_ID
1    admision
Name: ACTIVITY, dtype: object

show results

In [31]:
df_filter.groupby("ACTIVITY")["DIFFERENCE"].mean()

ACTIVITY
Aalta             0.000000e+00
admision          7.449733e+02
interconsulta     1.715403e+04
respuesta         1.834413e+04
rx exploracion    2.870873e+06
rx informe        2.085133e+04
rx solicitud      6.787239e+03
triaje            8.350999e+03
Name: DIFFERENCE, dtype: float64

## create minero

In [0]:
ouput_file= 'input_data_miner_all_mean_00_200_extended_time.csv'
create_minero_file(df_filter, ouput_file)

# Test all routes deterministic, parameters 00 resources 200, extended time

load and filter

In [23]:
location = 'drive/My Drive/data/prosae/testAllRoutesMean/res_simulacion_all_deterministic_mean_00_200_extended_time.csv'
df_load = load_data(location)
df_filter =  filter_bad_data(df_load)


   CASE_ID        ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR  DIFFERENCE
0        1        admision   161476390.0    161477120.0    NaN       730.0
1        1          triaje   161477120.0    161483290.0    NaN      6170.0
2        1    rx solicitud   161483290.0    161488641.0    NaN      5351.0
3        1  rx exploracion   161488641.0    161503386.0    NaN     14745.0
4        1      rx informe   161503386.0    161520427.0    NaN     17041.0
   CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR  DIFFERENCE
0        1  admision   161476390.0    161477120.0    NaN       730.0
       CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR   DIFFERENCE
54353    46134  admision   167476260.0            0.0    NaN -167476260.0
       CASE_ID  ACTIVITY  TIMESTAMP_IN  TIMESTAMP_OUT  COLOR   DIFFERENCE
54353    46134  admision   167476260.0            0.0    NaN -167476260.0
Len before prefilter: 8220
Len after prefilter: 7989


check

In [24]:
(df_filter.sort_values("TIMESTAMP_IN").groupby("CASE_ID")["ACTIVITY"].first() ).drop_duplicates()

CASE_ID
1    admision
Name: ACTIVITY, dtype: object

show results

In [25]:
df_filter.groupby("ACTIVITY")["DIFFERENCE"].mean()

ACTIVITY
Aalta                 0.0
admision            730.0
interconsulta     15454.0
respuesta         15192.0
rx exploracion    14745.0
rx informe        17041.0
rx solicitud       5351.0
triaje             6170.0
Name: DIFFERENCE, dtype: float64

create minero

In [0]:
ouput_file= 'input_data_miner_all_mean_determnistic_00_200_extended_time.csv'
create_minero_file(df_filter, ouput_file)